In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import normal
from keras.utils import plot_model
from deap import base, creator, tools, algorithms
from deap.benchmarks.tools import diversity, convergence, hypervolume
from functools import partial
import numpy as np
import random

Using TensorFlow backend.


In [2]:
def build_model(n_layers, input_dim, neurons, activation='sigmoid', initializer=None):
    if isinstance(neurons, list):
        assert len(neurons) == n_layers
    else:
        neurons = [neurons] * n_layers
        
    if initializer is None:
        # Uses normal initializer
        initializer = normal(mean=0, stddev=0.1, seed=13)
        
    model = Sequential()
    
    # Adds first hidden layer with input_dim parameter
    model.add(Dense(units=neurons[0], 
                    input_dim=input_dim, 
                    activation=activation,
                    kernel_initializer=initializer, 
                    name='hidden_layer'))
    
    
    # Adds output layer
    model.add(Dense(units=2, activation=activation, kernel_initializer=initializer, name='net_output'))
    
    # Compiles the model
    model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mean_squared_error'])
    
    return model

In [3]:
model = build_model(n_layers=1, input_dim=7, neurons=5)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def eval_function(model, chromosome):
    x_data = np.random.random((1, 7))
    net_output = model.predict(x_data)[0]
    return (net_output[0]*np.random.random_sample(), net_output[1]*np.random.random_sample(),)

In [5]:
def init_individual(cls, model):
    ind = cls(np.concatenate(
        (
            model.get_weights()[0].flatten(),
            model.get_weights()[2].flatten()
        )
    ).tolist())

    ind.shape_1 = model.get_weights()[0].shape
    ind.shape_2 = model.get_weights()[2].shape
    ind.features = None
    ind.weights = None

    return ind

CXPB = 0.9
N_GEN = 20
POP = 20

# Creating the appropriate type of the problem
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list,
               fitness=creator.FitnessMax, model=None)

toolbox = base.Toolbox()
history = tools.History()

toolbox.register("individual", init_individual,
                 creator.Individual, model=model)

# register the crossover operator
toolbox.register('mate', tools.cxTwoPoint)

# register the mutation operator
toolbox.register('mutate', tools.mutFlipBit, indpb=0.5)

# register the evaluation function
toolbox.register('evaluate', partial(eval_function, model))

# register NSGA-II multiobjective optimization algorithm
toolbox.register("select", tools.selNSGA2)

# instantiate the population
toolbox.register('population', tools.initRepeat,
                 list, toolbox.individual)

# maintain stats of the evolution
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('avg', np.mean)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('max', np.max)

logbook = tools.Logbook()
logbook.header = "gen", "evals", "std", "min", "avg", "max"

# create an initial population of N individuals
pop = toolbox.population(n=POP)
history.update(pop)

# object that contain the best individuals
hof = tools.ParetoFront()

# Evaluate the individuals with an invalid fitness
invalid_ind = [ind for ind in pop if not ind.fitness.valid]
fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)

for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit

# This is just to assign the crowding distance to the individuals
# no actual selection is done
pop = toolbox.select(pop, len(pop))
record = stats.compile(pop)
logbook.record(gen=0, evals=len(invalid_ind), **record)
print(logbook.stream)
hof.update(pop)

best_inds, best_inds_fitness = [], []

# Begin the generational process
for gen in range(1, N_GEN+1):
    # Vary the population
    offspring = tools.selTournamentDCD(pop, len(pop))
    offspring = [toolbox.clone(ind) for ind in offspring]

    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if random.random() <= CXPB:
            toolbox.mate(ind1, ind2)

        toolbox.mutate(ind1)
        toolbox.mutate(ind2)
        del ind1.fitness.values, ind2.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # add the best individual for each generation
    best_ind = tools.selBest(pop, 1)[0]
    best_inds.append(best_ind)
    best_inds_fitness.append(best_ind.fitness.values)

    # Select the next generation population
    pop = toolbox.select(pop + offspring, len(offspring))
    record = stats.compile(pop)
    logbook.record(gen=gen, evals=len(invalid_ind), **record)
    hof.update(pop)

print(logbook.stream)

gen	evals	std     	min       	avg    	max     
0  	20   	0.150455	0.00213751	0.27826	0.499812
1  	20   	0.148792	0.00213751	0.340318	0.499812
2  	20   	0.14902 	0.00213751	0.348065	0.499812
3  	20   	0.150344	0.00213751	0.356254	0.499812
4  	20   	0.144813	0.00213751	0.371361	0.499812
5  	20   	0.111353	0.00721352	0.397109	0.499812
6  	20   	0.142861	0.00721352	0.379489	0.503717
7  	20   	0.140688	0.00721352	0.381588	0.503717
8  	20   	0.128579	0.0179155 	0.388223	0.503717
9  	20   	0.127695	0.0179155 	0.389502	0.503717
10 	20   	0.129663	0.0179155 	0.387579	0.503717
11 	20   	0.141915	0.0179155 	0.380596	0.503717
12 	20   	0.139106	0.0179155 	0.383908	0.503717
13 	20   	0.139106	0.0179155 	0.383908	0.503717
14 	20   	0.139031	0.0179155 	0.38522 	0.503717
15 	20   	0.14644 	0.0179155 	0.379111	0.503717
16 	20   	0.151957	0.0179155 	0.372729	0.503717
17 	20   	0.151957	0.0179155 	0.372729	0.503717
18 	20   	0.145917	0.0179155 	0.378183	0.503717
19 	20   	0.145917	0.0179155 	0.378183	0.5

In [7]:
ind = toolbox.individual()

In [9]:
ind.__dict__

{'fitness': deap.creator.FitnessMax(()),
 'shape_1': (7, 5),
 'shape_2': (5, 2),
 'features': None,
 'weights': None}

In [15]:
ind

[0.017451664432883263,
 -0.022578200325369835,
 0.019674310460686684,
 0.038747698068618774,
 0.025138601660728455,
 0.005156669765710831,
 -0.010682681575417519,
 -0.12614494562149048,
 -0.023369936272501945,
 -0.10256432741880417,
 -0.05181825906038284,
 -0.03860422968864441,
 -0.13391904532909393,
 -0.06569492071866989,
 0.1440880447626114,
 0.08051245659589767,
 0.0773625299334526,
 0.015084986574947834,
 0.027785733342170715,
 -0.08993145823478699,
 0.13227465748786926,
 0.07975079864263535,
 0.08298309892416,
 0.015040106140077114,
 -0.09476090967655182,
 -0.0743572860956192,
 -0.006988647859543562,
 -0.06255804002285004,
 -0.1166984960436821,
 -0.01686728559434414,
 0.06776601076126099,
 -0.0345042422413826,
 0.17205838859081268,
 -0.02406460791826248,
 0.039534296840429306,
 0.017451664432883263,
 -0.022578200325369835,
 0.019674310460686684,
 0.038747698068618774,
 0.025138601660728455,
 0.005156669765710831,
 -0.010682681575417519,
 -0.12614494562149048,
 -0.02336993627250194